<a href="https://colab.research.google.com/github/fatemafaria142/Comparative-Analysis-of-Diverse-Large-Language-Models-in-Story-Generation/blob/main/Story_Generation_using_Starling_LM_7B_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00


#### **Dataset Link:** https://huggingface.co/datasets/AtlasUnified/atlas-storyteller

In [2]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("AtlasUnified/atlas-storyteller")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

### **Dataset structure**
* The dataset contains two different column.

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 5018
    })
})

In [4]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("ID:", data['id'])
    print("Story:", data['Story'])
    print("\n-----------------------------\n")

Data Point 1:
ID: seed_task_0
Story: In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They're coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan's blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment's hesitation, Ethan gathered his meager belongings and made his way to a hidden room beneath his apartment. The room

### **We will use just a small subset of the data for this training example**

In [6]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(300))

In [7]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['id', 'Story'],
        num_rows: 300
    })
})

In [26]:
def create_prompt(sample):
     # Use a predefined template for instructions
    prompt = "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! "
    prompt += sample['Story']
    single_turn_prompt = f"GPT4 Correct User: {prompt}<|end_of_turn|> GPT4 Correct Assistant: "
    return single_turn_prompt

In [27]:
create_prompt(instruct_tune_dataset["train"][0])

'GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They\'re coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why w

In [28]:
create_prompt(instruct_tune_dataset["train"][1])

'GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! The sky stretched out before them, an endless expanse of blue pierced only by fluffy white clouds. The air crackled with the anticipation of action as the squadron gathered at the airfield, their sleek fighter jets gleaming under the noon sun. Among the pilots was Captain Joshua Bennett, a seasoned veteran with eyes as sharp as a hawk\'s, and a heart that beat with the rhythm of war.  Set in the midst of a raging conflict, the story takes place during the height of World War II. Captain Bennett, a fearless and daring leader, had earned his reputation through countless successful missions. He was known for his audacious maneuvers and masterful control of aircraft, traits that instilled confidence in every member of h

In [29]:
create_prompt(instruct_tune_dataset["train"][2])

'GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the midst of a bustling metropolis teeming with skyscrapers that seemingly touched the heavens, nestled a small and unassuming garage. It belonged to Ethan, a young and brilliant engineer consumed by his insatiable thirst for innovation and adventure. With grease-stained hands and a mind full of ideas, he spent every waking hour tinkering and crafting marvelous inventions that could turn dreams into reality.  One fateful afternoon, while rummaging through a box filled with ancient blueprints, Ethan stumbled upon a peculiar schematic. It depicted a colossal machine, a menacing giant robot with gleaming steel limbs and an aura of unstoppable power. Intrigued and fascinated, he couldn\'t resist the temptation to bri

In [30]:
create_prompt(instruct_tune_dataset["train"][3])

"GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the heart of a once-thriving metropolis, now reduced to a desolate wasteland, a lone figure emerged from the shadows. Leather-clad and armed to the teeth, his name was Gabriel. Standing at the precipice of an abandoned street, he surveyed the remnants of what was once a bustling city. Hollowed buildings loomed like guardians of a forgotten time, their broken windows reflecting the glimmers of an apocalyptic twilight.  It had been years since the outbreak, the virus that turned ordinary people into ravenous zombies. Gabriel, a former army officer, had become a symbol of hope in this desolate world. With his unwavering determination and unparalleled combat skills, he had earned the respect of fellow survivors. But 

In [31]:
create_prompt(instruct_tune_dataset["train"][4])

'GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with. Tall and lean, with a sharp jawline and piercing blue eyes, he exuded an aura of unwavering determination. The city had become his battleground, and the criminals feared his name.  The whispers on the streets spoke of a notorious criminal mastermind known as "The Phantom." No one had ever seen his face, and his crimes left the police baffled. Like a phantom, he slipped through their fingers, leaving no trace behind. Detective Anderson was obsessed with catching him, a relentless pursuit that consumed his every waking moment.  Late one stormy evening, as the rain poured mercilessly, Detective Anders

In [32]:
create_prompt(instruct_tune_dataset["train"][5])

"GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the crumbling metropolis of Neo-Arkadia, a city teetering on the brink of chaos, rival gangs waged war for control over scarce resources. Dilapidated skyscrapers stood tall, their shattered windows reflecting the desolation that shrouded the once-thriving urban landscape. Among this dystopian backdrop, two formidable gangs emerged as the most powerful contenders: the Razor Wings and the Shadow Serpents.  The city's unmistakable staleness hung in the air as the sun dipped below the horizon, casting long shadows across the cracked pavement. The Red Sector, a territory claimed by the Shadow Serpents, pulsated with an otherworldly energy. Led by the enigmatic and ruthless Viper, the gang thrived in the darkness, leav

In [33]:
create_prompt(instruct_tune_dataset["train"][6])

"GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! Amidst the inky blackness of the night, a seamless curtain of shadows unfurled itself over the dense wilderness. In the heart of a war-torn nation, nestled between towering mountains and treacherous valleys, lay a remote enemy stronghold. It was here that the fate of an entire nation teetered on the precipice of destruction. Behind the menacing gates and fortified walls, the nefarious General Viktor Verenov reigned supreme, orchestrating his malevolent plans in secret.  In this harrowing landscape, emerged our indomitable hero, Captain Ethan Reynolds, an elite operative trained in the art of covert warfare. With a lean and chiseled physique, Ethan's eyes burned with a fierce determination, his every muscle honed to 

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [35]:
mode_id = "berkeley-nest/Starling-LM-7B-alpha"

In [36]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [37]:
tokenizer = AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently:**

In [38]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [41]:
prompt = """ GPT4 Correct User: "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! """
prompt += '''Amidst the inky blackness of the night, a seamless curtain of shadows unfurled itself over the dense wilderness. In the heart of a war-torn nation, nestled between towering mountains and treacherous valleys, lay a remote enemy stronghold. It was here that the fate of an entire nation teetered on the precipice of destruction. <|end_of_turn|> '''
prompt += "GPT4 Correct Assistant:"

In [42]:
print(prompt)

 GPT4 Correct User: "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! Amidst the inky blackness of the night, a seamless curtain of shadows unfurled itself over the dense wilderness. In the heart of a war-torn nation, nestled between towering mountains and treacherous valleys, lay a remote enemy stronghold. It was here that the fate of an entire nation teetered on the precipice of destruction. <|end_of_turn|> GPT4 Correct Assistant:


In [43]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<s>  GPT4 Correct User: "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! Amidst the inky blackness of the night, a seamless curtain of shadows unfurled itself over the dense wilderness. In the heart of a war-torn nation, nestled between towering mountains and treacherous valleys, lay a remote enemy stronghold. It was here that the fate of an entire nation teetered on the precipice of destruction. <|end_of_turn|>  GPT4 Correct Assistant: As the night progressed, the only light came from the burning embers of a nearby campfire, flickering against the trees and casting eerie, twisted shadows. Amidst the darkness, the sound of marching boots and heavy breathing could be heard from the hidden enemy troops, preparing for a surprise attack on a nearby village. One by one, the soldiers cre

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [44]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [45]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [46]:
from transformers import TrainingArguments
output_model= "Starling_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [47]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [48]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.623800
20,1.439000
30,1.338200
40,1.321700
50,1.310000
60,1.326300
70,1.326200
80,1.291200
90,1.246500
100,1.308700


TrainOutput(global_step=150, training_loss=1.3392448234558105, metrics={'train_runtime': 1189.4506, 'train_samples_per_second': 0.504, 'train_steps_per_second': 0.126, 'total_flos': 1.3112665178112e+16, 'train_loss': 1.3392448234558105, 'epoch': 0.08})

### **Save the model**

In [49]:
trainer.save_model("Starling_instruct_generation")

In [50]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [51]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [52]:
prompt = "GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! "
prompt += "In the crumbling metropolis of Neo-Arkadia, a city teetering on the brink of chaos, rival gangs waged war for control over scarce resources. Dilapidated skyscrapers stood tall, their shattered windows reflecting the desolation that shrouded the once-thriving urban landscape. <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the crumbling metropolis of Neo-Arkadia, a city teetering on the brink of chaos, rival gangs waged war for control over scarce resources. Dilapidated skyscrapers stood tall, their shattered windows reflecting the desolation that shrouded the once-thriving urban landscape. <|end_of_turn|>  GPT4 Correct Assistant: Among the ruins of Neo-Arkadia, a lone survivor named Lila eked out an existence. With her unwavering determination and a relentless spirit, she sought to forge a new life for herself in the midst of the city's decline. Lila was a scavenger, rummaging through the rubble for anything she could repurpose or sell to make a living. Her days were spent unearthing precious artifacts, scraps of clothing, and 

### **Example No:2**

In [53]:
prompt = "GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! "
prompt += "In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with. Tall and lean, with a sharp jawline and piercing blue eyes, he exuded an aura of unwavering determination. The city had become his battleground, and the criminals feared his name. <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with. Tall and lean, with a sharp jawline and piercing blue eyes, he exuded an aura of unwavering determination. The city had become his battleground, and the criminals feared his name. <|end_of_turn|>  GPT4 Correct Assistant: One fateful evening, as the city shimmered with neon lights and the echo of sirens filled the air, John received a tip from a confidential informant, a man known only as "The Ghost." The information was delicate and potentially explosive—a notorious crime lord named Vittorio Russo was planning a heist unlike any the city had ever seen. To steal a priceless artifact from the muse

### **Example No:3**

In [54]:
prompt = "GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! "
prompt += "The sky stretched out before them, an endless expanse of blue pierced only by fluffy white clouds. The air crackled with the anticipation of action as the squadron gathered at the airfield, their sleek fighter jets gleaming under the noon sun. <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! The sky stretched out before them, an endless expanse of blue pierced only by fluffy white clouds. The air crackled with the anticipation of action as the squadron gathered at the airfield, their sleek fighter jets gleaming under the noon sun. <|end_of_turn|>  GPT4 Correct Assistant: Major Sam Thompson, the team leader, stood at the head of the formation, his steely gaze scanning the horizon. Beside him, the team's newest member, Lieutenant Olivia Turner, a brilliant aerial tactician, eagerly awaited her first mission. Sam couldn't help but notice the raw determination in her eyes—an ember ready to ignite. 

Their mission was to intercept and neutralize a rogue drone that had deviated from its original course and